### Import libraries

In [1]:
import sys; sys.path.insert(0, '..')
import os

import ee
import numpy as np
import geemap
from osgeo import gdal

import settings_BID
import utils

ee.Initialize()

### Define parameters

In [57]:
# [lat, lon]
sauce_center = [-34.83326008, -55.05417706]
center_point = ee.Geometry.Point(sauce_center[1], sauce_center[0])
# [[lon, lat]]
sauce_zone = [[-55.16629529644229, -34.7494869239046],
              [-55.02038312603214, -34.7494869239046],
              [-55.02038312603214, -34.868725532230165],
              [-55.16629529644229, -34.868725532230165],
              [-55.16629529644229, -34.7494869239046]]
sauce_geometry = ee.Geometry.Polygon(sauce_zone)
#Dates to filter
start = ee.Date('2016-01-01')
finish = ee.Date('2021-01-31')
# RGB + near infrared
bands = ['B5', 'B4', 'B3', 'B2']
RGB = ['B4', 'B3', 'B2']

### Define some functions

In [22]:
def get_image_date(image):
    return ee.Feature(None, {'date': image.date().format('YYYY-MM-dd')})

def clip_image(image):
    return image.clip(sauce_geometry)

def ymdList(imgcol):
    def iter_func(image, newlist):
        date = ee.Number.parse(image.date().format("YYYYMMdd"));
        newlist = ee.List(newlist);
        return ee.List(newlist.add(date).sort())
    ymd = imgcol.iterate(iter_func, ee.List([]))
    return list(ee.List(ymd).reduce(ee.Reducer.frequencyHistogram()).getInfo().keys())

### Get landsat image dates

In [30]:
landsat_collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(sauce_geometry) \
    .filterDate(start, finish)

count = str(landsat_collection.size().getInfo())
print("Landsat-8 count: {}".format(count))
landsat_dates = ymdList(landsat_collection)
#clipped_collection = landsat_collection.map(clip_image)

Landsat-8 count: 180


### Get Sentinel-2 image dates

In [31]:
sentinel_collection = ee.ImageCollection('COPERNICUS/S2_SR') \
    .filterBounds(sauce_geometry) \
    .filterDate(start, finish)

count = str(sentinel_collection.size().getInfo())
print("Sentinel-2 count: {}".format(count))
sentinel_dates = ymdList(sentinel_collection)
#clipped_collection = sentinel_collection.map(clip_image)

Sentinel-2 count: 311


### Get matching dates

In [45]:
from datetime import datetime

matching_dates = []
for l_d in landsat_dates:
    if l_d in sentinel_dates:
        matching_dates.append(ee.Date(datetime.strptime(l_d, "%Y%m%d")))
print("Matching days count: {}".format(len(matching_dates)))

Matching days count: 29


### Get landsat less cloudy date

In [70]:
cloud_scores = {}

for day in matching_dates:
    print(day.format("YYYY-MM-dd").getInfo())
    
    landsat_day = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
                    .filterBounds(sauce_geometry) \
                    .filterDate(day, day.advance(1, 'day'))
    
    landsat_day = landsat_day.first()
    landsat_day = landsat_day.clip(sauce_geometry)
    landsat_cloud_cover_score = landsat_day.get('CLOUD_COVER')
    landsat_cloud_cover_score = ee.Feature(None, {'score': landsat_cloud_cover_score}).getInfo()["properties"]["score"]
    cloud_scores[str(day.format("YYYY-MM-dd").getInfo())] = landsat_cloud_cover_score

    #     sentinel_day = ee.ImageCollection('COPERNICUS/S2_SR') \
    #     .filterBounds(sauce_geometry) \
    #     .filterDate(day, day.advance(1, 'day'))
    #     sentinel_day = sentinel_day.first()
    #     sentinel_day = sentinel_day.clip(sauce_geometry)
    #     sentinel_cloud_cover_score = sentinel_day.get('CLOUD_COVER')
    #     sentinel_cloud_cover_score = ee.Feature(None, {'score': sentinel_cloud_cover_score}).getInfo()["properties"]["score"]
    #     print(sentinel_cloud_cover_score)
    #     print("---------")
best_date = ee.Date(datetime.strptime(min(cloud_scores, key=cloud_scores.get), "%Y-%m-%d"))

2019-01-01
2019-01-26
2019-02-02
2019-02-27
2019-03-22
2019-04-16
2019-04-23
2019-05-18
2019-07-05
2019-08-06
2019-08-29
2019-09-23
2019-09-30
2019-10-25
2019-11-17
2019-12-12
2020-01-13
2020-02-05
2020-03-01
2020-03-08
2020-04-02
2020-04-25
2020-05-20
2020-05-27
2020-07-14
2020-08-15
2020-10-02
2020-10-27
2020-11-28


### Get landsat and sentinel data from that date

In [76]:
landsat_day = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
                    .filterBounds(sauce_geometry) \
                    .filterDate(best_date, best_date.advance(1, 'day'))
    
landsat_day = landsat_day.first()
landsat_day = landsat_day.clip(sauce_geometry)
print(landsat_day.getDownloadURL())


sentinel_day = ee.ImageCollection('COPERNICUS/S2_SR') \
                .filterBounds(sauce_geometry) \
                .filterDate(best_date, best_date.advance(1, 'day'))
sentinel_day = sentinel_day.first()
sentinel_day = sentinel_day.clip(sauce_geometry)
nameOfBands = sentinel_day.bandNames().getInfo()
nameOfBands.remove("B6")
nameOfBands.remove("B7")
nameOfBands.remove("B8")
nameOfBands.remove("B9")
nameOfBands.remove("B11")
nameOfBands.remove("B12")
sentinel_day = sentinel_day.select(nameOfBands)
print(sentinel_day.getDownloadURL())

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/61e79a04a6429a208866747afecb2085-f7a1f3f6afceaca05c1035d098cf3f2d:getPixels
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/22f52bd539fe688ab72a705efafc4ca7-964b4a54b275e2e66c012c03e0a612e6:getPixels
